In [1]:
from google.colab import files
uploaded = files.upload()

Saving upload_zip.zip to upload_zip.zip


In [2]:
!unzip -q upload_zip.zip

In [3]:
# dowanload the dogs_and_cats dataset
!wget https://nextcloud.rickdgray.com/index.php/s/n2ZX7n2er4z3PpP/download/dogs_and_cats.zip

!unzip -q dogs_and_cats.zip -d data

--2024-02-20 04:11:11--  https://nextcloud.rickdgray.com/index.php/s/n2ZX7n2er4z3PpP/download/dogs_and_cats.zip
Resolving nextcloud.rickdgray.com (nextcloud.rickdgray.com)... 45.24.51.109
Connecting to nextcloud.rickdgray.com (nextcloud.rickdgray.com)|45.24.51.109|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 882765995 (842M) [application/zip]
Saving to: ‘dogs_and_cats.zip’

dogs_and_cats.zip   100%[===================>] 841.87M  11.9MB/s    in 81s     

2024-02-20 04:12:32 (10.4 MB/s) - ‘dogs_and_cats.zip’ saved [882765995/882765995]



In [4]:
%pylab inline
import torch
import sys
sys.path.append('..')
sys.path.append('../..')
from data import load
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('device = ', device)

Populating the interactive namespace from numpy and matplotlib
device =  cuda


In [5]:
class ConvNet(torch.nn.Module):
    class Block(torch.nn.Module):
        def __init__(self, n_input, n_output, stride=1):
            super().__init__()
            self.net = torch.nn.Sequential(
              torch.nn.Conv2d(n_input, n_output, kernel_size=3, padding=1, stride=stride, bias=False), # set bias to false since adding batch norm. after conv.
              torch.nn.BatchNorm2d(n_output), # Add batch normalization after convolution (takes argument num_features which is number of output channels from conv. layer before)
              torch.nn.ReLU(),                # can also change batchnorm to layernorm or similar normalization
              torch.nn.Conv2d(n_output, n_output, kernel_size=3, padding=1, bias=False), # Remember to set bias to false
              torch.nn.BatchNorm2d(n_output), # Add again after convolution
              torch.nn.ReLU()
            )

        def forward(self, x):
            return self.net(x)

    def __init__(self, layers=[32,64,128], n_input_channels=3):
        super().__init__()
        L = [torch.nn.Conv2d(n_input_channels, 32, kernel_size=7, padding=3, stride=2, bias=False),
             torch.nn.BatchNorm2d(32), # Add layer after each of convolutions, remember to sitch between training and eval
             torch.nn.ReLU(),
             torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
            ]
        c = 32
        for l in layers:
            L.append(self.Block(c, l, stride=2))
            c = l
        self.network = torch.nn.Sequential(*L)
        self.classifier = torch.nn.Linear(c, 1)

    def forward(self, x):
        # Compute the features
        z = self.network(x)
        # Global average pooling
        z = z.mean(dim=[2,3])
        # Classify
        return self.classifier(z)[:,0]

net = ConvNet()
net.train() #NEED TO DO WHEN TRAINING, training mode collects batch stats and computes running average over batch statistics
print( net.training )
net.eval() # WHEN DONE WITH TRAINING, eval mode only uses this running average, does not update any internal running avg or compute actual statistics
print( net.training )

True
False
